In [1]:
!python -m pip install git+https://github.com/carnotresearch/cr-vision.git


  Cloning https://github.com/carnotresearch/cr-vision.git to /tmp/pip-req-build-ncamz39o
  Running command git clone -q https://github.com/carnotresearch/cr-vision.git /tmp/pip-req-build-ncamz39o
     |████████████████████████████████| 2.3MB 9.0MB/s 
     |████████████████████████████████| 204kB 32.9MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 9.2MB 37.6MB/s 
     |████████████████████████████████| 2.8MB 56.5MB/s 
     |████████████████████████████████| 563kB 48.0MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
     |████████████████████████████████| 122kB 59.7MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
  Created wheel for cr-vision: filename=cr_vision-0.1.0-cp37-none-any.whl size=80196 sha256=7ec4d3ad6c3f168bd989bb89dde345aa325cf471bd0ce1894bb8be4bcebe2eae
  Stored in directory: /tmp/pip-ephem-wheel-cache-bspcapw4/wheels/22/29/66/6318edcf35e0e833a

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp /content/drive/MyDrive/work/cr-vision/experiments/cs/sda/sda.py .

In [3]:
from pathlib import Path
from cr.vision.io import images_from_dir
import sda

In [6]:
!cp  /content/drive/MyDrive/datasets/birds/birds_subset_5000/training.zip .

In [ ]:
!unzip training.zip

In [8]:
!cp  /content/drive/MyDrive/datasets/birds/birds_subset_5000/validation.zip .

In [ ]:
!unzip validation.zip

In [5]:
rootdir = Path('.')
training_set = images_from_dir(rootdir / 'training', size=1200)
validation_set = images_from_dir(rootdir / 'validation', size=400)


training/025.Pelagic_Cormorant_Pelagic_Cormorant_0022_23802.jpg, (375, 500)
shape after conversion (375, 500, 3)
training/063.Ivory_Gull_Ivory_Gull_0040_49180.jpg, (326, 500)
shape after conversion (326, 500, 3)


In [6]:
print(f"training set: {training_set.shape}")
print(f"validation set: {validation_set.shape}")

training set: (1200, 256, 256, 3)
validation set: (400, 256, 256, 3)


In [7]:
training_set.shape

(1200, 256, 256, 3)

In [8]:
validation_set.shape

(400, 256, 256, 3)

In [9]:
batch_size = 32

In [10]:
train_gen = sda.augment_training_set(training_set, batch_size = batch_size)

Preparing augmented training set generator.


In [11]:
input_shape = training_set[0].shape
input_shape

(256, 256, 3)

In [12]:
patch_size = 16

In [13]:
compression_ratio = 16

In [14]:
models = sda.build_models(input_shape,
    patch_size=patch_size,
    compression_ratio=compression_ratio)

In [15]:
autoencoder = models.autoencoder
autoencoder.summary()

Model: "FCSDA_Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 256, 256, 3)]     0         
_________________________________________________________________
FCSDA_Encoder (Functional)   (None, 16, 16, 48)        36864     
_________________________________________________________________
FCSDA_Decoder (Functional)   (None, 256, 256, 3)       38723     
Total params: 75,587
Trainable params: 38,595
Non-trainable params: 36,992
_________________________________________________________________


In [16]:
encoder = models.encoder
encoder.summary()

Model: "FCSDA_Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 256, 256, 3)]     0         
_________________________________________________________________
sensor (Conv2D)              (None, 16, 16, 48)        36864     
Total params: 36,864
Trainable params: 0
Non-trainable params: 36,864
_________________________________________________________________


In [17]:
decoder = models.decoder
decoder.summary()

Model: "FCSDA_Decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoded_input (InputLayer)   [(None, 16, 16, 48)]      0         
_________________________________________________________________
layer-1 (Conv2D)             (None, 16, 16, 16)        784       
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 16, 16, 16)        64        
_________________________________________________________________
layer-2 (Conv2D)             (None, 16, 16, 48)        816       
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 16, 16, 48)        192       
_________________________________________________________________
layer-3 (Conv2DTranspose)    (None, 256, 256, 3)       36867     
Total params: 38,723
Trainable params: 38,595
Non-trainable params: 128
_______________________________________________

In [18]:
sda.compile_model(autoencoder)

Compiling model for training.


In [19]:
callbacks = sda.build_callbacks()


Preparing callbacks for model training.


In [20]:
n_train = training_set.shape[0]
n_train

1200

In [21]:
steps_per_epoch = int(n_train / batch_size)
steps_per_epoch


37

In [22]:
epochs=50

In [23]:
history = sda.fit_model(autoencoder, train_gen,
    validation_set, callbacks,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs)

Initiating model training for 50 epochs.
37 50
(400, 256, 256, 3)
Epoch 1/50
37/37 [==============================] - 59s 744ms/step - loss: 0.0587 - val_loss: 0.0884

Epoch 00001: val_loss improved from inf to 0.08836, saving model to /content/drive/MyDrive/work/cr-vision/experiments/cs/sda/checkpoint.hdf5
Epoch 2/50
37/37 [==============================] - 27s 740ms/step - loss: 0.0247 - val_loss: 0.0612

Epoch 00002: val_loss improved from 0.08836 to 0.06123, saving model to /content/drive/MyDrive/work/cr-vision/experiments/cs/sda/checkpoint.hdf5
Epoch 3/50
37/37 [==============================] - 27s 725ms/step - loss: 0.0194 - val_loss: 0.0359

Epoch 00003: val_loss improved from 0.06123 to 0.03595, saving model to /content/drive/MyDrive/work/cr-vision/experiments/cs/sda/checkpoint.hdf5
Epoch 4/50
37/37 [==============================] - 26s 721ms/step - loss: 0.0142 - val_loss: 0.0260

Epoch 00004: val_loss improved from 0.03595 to 0.02600, saving model to /content/drive/MyDrive/

In [24]:
sda.save_model(models.autoencoder, 'autoencoder', compression_ratio)
sda.save_model(models.encoder, 'encoder', compression_ratio)
sda.save_model(models.decoder, 'decoder', compression_ratio)

Saving autoencoder at compression ration 16 in tensorflow SavedModel format.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/work/cr-vision/experiments/cs/sda/saved_model_autoencoder_16/assets
Saving encoder at compression ration 16 in tensorflow SavedModel format.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/work/cr-vision/experiments/cs/sda/saved_model_encoder_16/assets
Saving decoder at compression ration 16 in tensorflow SavedModel format.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/work/cr-vision/experiments/cs/sda/saved_model_decoder_16/assets


In [25]:
history_file = f'{sda.GD_EXP_DIR}_history_cr_{compression_ratio}.json'
history_file

'/content/drive/MyDrive/work/cr-vision/experiments/cs/sda_history_cr_16.json'

In [26]:
import pandas as pd

In [27]:
hist_df = pd.DataFrame(history.history)

In [29]:
with open(history_file, mode='w') as f:
        hist_df.to_json(f)